# Начальная инициализация

In [1]:
import datetime
print(datetime.datetime.now().strftime('%d-%B-%Y %H:%M:%S'))

26-May-2023 18:36:03


In [2]:
PATH = '/kaggle/input/ml-3-lab/'
# PATH = ''

In [3]:
# Подключение библиотек

# !pip install torch torchvision torchaudio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils

# !pip install scikit-learn
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import ParameterGrid

import torch.optim as optim
# !pip install pandas
import pandas as pd
import numpy as np

# import re
# !pip install nltk
# import nltk
# nltk.download('punkt')
# from string import punctuation

# %pip install gensim
import gensim
from gensim.models import Word2Vec
from gensim.models import FastText

import os
import random

In [4]:
# Подключение вычислений на видеокарту, если доступна
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)
# device = torch.device("cpu")
print(device)

cuda:0


In [5]:
cpu_count = os.cpu_count()

num_workers = cpu_count if device == torch.device("cpu") else 0
num_workers, cpu_count

(0, 2)

In [6]:
random_state = 42

torch.use_deterministic_algorithms(True)

%env CUBLAS_WORKSPACE_CONFIG=:4096:8
%env PYTHONHASHSEED=42

def set_random_state(random_state):
    torch.manual_seed(random_state)
    random.seed(random_state)
    np.random.seed(random_state)

    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(random_state)
        torch.cuda.manual_seed(random_state)

        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
set_random_state(42)

env: CUBLAS_WORKSPACE_CONFIG=:4096:8
env: PYTHONHASHSEED=42


# Модели и методы для обучения и тестирования

In [7]:
def word_averaging(model, words):
    all_words, mean = set(), []

    for word in words:
        mean.append(model.wv[word])
        all_words.add(word)

    if not mean:
        return np.zeros(model.vector_size)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)

    return mean


def word_averaging_list(model, text_list):
    return np.vstack([word_averaging(model, comment_text) for comment_text in text_list])

In [8]:
def word_averaging_pre_trained(model, words):
  all_words, mean = set(), []

  for word in words:
    if word in model.key_to_index:
      mean.append(model[model.key_to_index[word]])
      all_words.add(word)

  if not mean:
    return np.zeros(model.vector_size)

  mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)

  return mean


def word_averaging_list_pre_trained(model, text_list):
  return np.vstack([word_averaging_pre_trained(model, comment_text) for comment_text in text_list])

In [2]:
print(datetime.datetime.now().strftime('%d-%B-%Y %H:%M:%S'))

26-May-2023 18:36:16


## Предобработка текста

In [10]:
train = pd.read_csv(PATH + 'X_y_train.csv', sep=';')
test = pd.read_csv(PATH + 'X_y_test.csv', sep=';')

X_train = [el[0] for el in train[['Text']].values]
X_test = [el[0] for el in test[['Text']].values]
y_train = [el[0] for el in train[['Class']].replace(-1, 0).values]
y_test = [el[0] for el in test[['Class']].replace(-1, 0).values]

# Create a X_token file one time!

# def tokenize(text):

#   text_token = nltk.word_tokenize(text)
#   text_word = [el.lower() for el in text_token if el not in punctuation]
#   return text_word

# X_train_token = [tokenize(t) for t in X_train]
# X_test_token = [tokenize(t) for t in X_test]


# with open('X_train_token.txt', mode='w+') as file:
#     for sentence in X_train_token:
#         print(*sentence, file=file)

X_token = [sentence.split() for sentence in open(PATH + 'X_token.txt', mode='r')]
X_train_token = [sentence.split() for sentence in open(PATH + 'X_train_token.txt', mode='r')]
X_test_token = [sentence.split() for sentence in open(PATH + 'X_test_token.txt', mode='r')]

Функции для работы предобученных моделей RusVectores

In [11]:
import sys
import os
!pip install wget
import wget
import re
!pip install ufal.udpipe
from ufal.udpipe import Model, Pipeline

"""
Этот скрипт принимает на вход необработанный русский текст 
(одно предложение на строку или один абзац на строку).
Он токенизируется, лемматизируется и размечается по частям речи с использованием UDPipe.
На выход подаётся последовательность разделенных пробелами лемм с частями речи 
("зеленый_ADJ трамвай_NOUN").
Их можно непосредственно использовать в моделях с RusVectōrēs (https://rusvectores.org).

Примеры запуска:
echo 'Мама мыла раму.' | python3 rus_preprocessing_udpipe.py
zcat large_corpus.txt.gz | python3 rus_preprocessing_udpipe.py | gzip > processed_corpus.txt.gz
"""


def num_replace(word):
    newtoken = "x" * len(word)
    return newtoken


def clean_token(token, misc):
    """
    :param token:  токен (строка)
    :param misc:  содержимое поля "MISC" в CONLLU (строка)
    :return: очищенный токен (строка)
    """
    out_token = token.strip().replace(" ", "")
    if token == "Файл" and "SpaceAfter=No" in misc:
        return None
    return out_token


def clean_lemma(lemma, pos):
    """
    :param lemma: лемма (строка)
    :param pos: часть речи (строка)
    :return: очищенная лемма (строка)
    """
    out_lemma = lemma.strip().replace(" ", "").replace("_", "").lower()
    if "|" in out_lemma or out_lemma.endswith(".jpg") or out_lemma.endswith(".png"):
        return None
    if pos != "PUNCT":
        if out_lemma.startswith("«") or out_lemma.startswith("»"):
            out_lemma = "".join(out_lemma[1:])
        if out_lemma.endswith("«") or out_lemma.endswith("»"):
            out_lemma = "".join(out_lemma[:-1])
        if (
            out_lemma.endswith("!")
            or out_lemma.endswith("?")
            or out_lemma.endswith(",")
            or out_lemma.endswith(".")
        ):
            out_lemma = "".join(out_lemma[:-1])
    return out_lemma


def list_replace(search, replacement, text):
    search = [el for el in search if el in text]
    for c in search:
        text = text.replace(c, replacement)
    return text


def unify_sym(text):  # принимает строку в юникоде
    text = list_replace(
        "\u00AB\u00BB\u2039\u203A\u201E\u201A\u201C\u201F\u2018\u201B\u201D\u2019",
        "\u0022",
        text,
    )

    text = list_replace(
        "\u2012\u2013\u2014\u2015\u203E\u0305\u00AF", "\u2003\u002D\u002D\u2003", text
    )

    text = list_replace("\u2010\u2011", "\u002D", text)

    text = list_replace(
        "\u2000\u2001\u2002\u2004\u2005\u2006\u2007\u2008\u2009\u200A\u200B\u202F\u205F\u2060\u3000",
        "\u2002",
        text,
    )

    text = re.sub("\u2003\u2003", "\u2003", text)
    text = re.sub("\t\t", "\t", text)

    text = list_replace(
        "\u02CC\u0307\u0323\u2022\u2023\u2043\u204C\u204D\u2219\u25E6\u00B7\u00D7\u22C5\u2219\u2062",
        ".",
        text,
    )

    text = list_replace("\u2217", "\u002A", text)

    text = list_replace("…", "...", text)

    text = list_replace("\u2241\u224B\u2E2F\u0483", "\u223D", text)

    text = list_replace("\u00C4", "A", text)  # латинская
    text = list_replace("\u00E4", "a", text)
    text = list_replace("\u00CB", "E", text)
    text = list_replace("\u00EB", "e", text)
    text = list_replace("\u1E26", "H", text)
    text = list_replace("\u1E27", "h", text)
    text = list_replace("\u00CF", "I", text)
    text = list_replace("\u00EF", "i", text)
    text = list_replace("\u00D6", "O", text)
    text = list_replace("\u00F6", "o", text)
    text = list_replace("\u00DC", "U", text)
    text = list_replace("\u00FC", "u", text)
    text = list_replace("\u0178", "Y", text)
    text = list_replace("\u00FF", "y", text)
    text = list_replace("\u00DF", "s", text)
    text = list_replace("\u1E9E", "S", text)

    currencies = list(
        "\u20BD\u0024\u00A3\u20A4\u20AC\u20AA\u2133\u20BE\u00A2\u058F\u0BF9\u20BC\u20A1\u20A0\u20B4\u20A7\u20B0\u20BF\u20A3\u060B\u0E3F\u20A9\u20B4\u20B2\u0192\u20AB\u00A5\u20AD\u20A1\u20BA\u20A6\u20B1\uFDFC\u17DB\u20B9\u20A8\u20B5\u09F3\u20B8\u20AE\u0192"
    )

    alphabet = list(
        '\t\n\r абвгдеёзжийклмнопрстуфхцчшщьыъэюяАБВГДЕЁЗЖИЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ,.[]{}()=+-−*&^%$#@!?~;:0123456789§/\|"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ '
    )

    alphabet.append("'")

    allowed = set(currencies + alphabet)

    cleaned_text = [sym for sym in text if sym in allowed]
    cleaned_text = "".join(cleaned_text)

    return cleaned_text


def process(pipeline, text="Строка", keep_pos=True, keep_punct=False):
    # Если частеречные тэги не нужны (например, их нет в модели), выставьте pos=False
    # в этом случае на выход будут поданы только леммы
    # По умолчанию знаки пунктуации вырезаются. Чтобы сохранить их, выставьте punct=True

    entities = {"PROPN"}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [line for line in processed.split("\n") if not line.startswith("#")]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split("\t") for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        token = clean_token(token, misc)
        lemma = clean_lemma(lemma, pos)
        if not lemma or not token:
            continue
        if pos in entities:
            if "|" not in feats:
                tagged_propn.append("%s_%s" % (lemma, pos))
                continue
            morph = {el.split("=")[0]: el.split("=")[1] for el in feats.split("|")}
            if "Case" not in morph or "Number" not in morph:
                tagged_propn.append("%s_%s" % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph["Case"]
                mem_number = morph["Number"]
            if morph["Case"] == mem_case and morph["Number"] == mem_number:
                memory.append(lemma)
                if "SpacesAfter=\\n" in misc or "SpacesAfter=\s\\n" in misc:
                    named = False
                    past_lemma = "::".join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + "_PROPN")
            else:
                named = False
                past_lemma = "::".join(memory)
                memory = []
                tagged_propn.append(past_lemma + "_PROPN")
                tagged_propn.append("%s_%s" % (lemma, pos))
        else:
            if not named:
                if (
                    pos == "NUM" and token.isdigit()
                ):  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append("%s_%s" % (lemma, pos))
            else:
                named = False
                past_lemma = "::".join(memory)
                memory = []
                tagged_propn.append(past_lemma + "_PROPN")
                tagged_propn.append("%s_%s" % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split("_")[1] != "PUNCT"]
    if not keep_pos:
        tagged_propn = [word.split("_")[0] for word in tagged_propn]
    return tagged_propn


# URL of the UDPipe model
udpipe_model_url = "https://rusvectores.org/static/models/udpipe_syntagrus.model"
udpipe_filename = PATH + udpipe_model_url.split("/")[-1]

if not os.path.isfile(udpipe_filename):
    print("UDPipe model not found. Downloading...", file=sys.stderr)
    wget.download(udpipe_model_url)

print("\nLoading the model...", file=sys.stderr)
model = Model.load(udpipe_filename)
process_pipeline = Pipeline(
    model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu"
)

  Preparing metadata (setup.py) ... - done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=12f96035523027d541581121106361d1a8aece6b2ebaaf0e8b1c2cf1ee2be14e
  Stored in directory: /root/.cache/pip/wheels/e1/e8/db/ebe4dcd7d7d11208c1e4e4ef246cea4fcc8d463c93405a6555
Successfully built wget
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 936.6/936.6 kB 32.5 MB/s eta 0:00:00



Loading the model...


In [3]:
print(datetime.datetime.now().strftime('%d-%B-%Y %H:%M:%S'))

26-May-2023 18:36:43


# Word2vec-признаки, обученные самостоятельно

## Обучение модели word2vec на полученном наборе данных

In [13]:
w2v_model = Word2Vec(sentences=X_token, vector_size=300, window=5, min_count=1, workers=cpu_count, seed=random_state)
w2v_model.build_vocab(X_token)
w2v_model.train(X_token, total_examples=w2v_model.corpus_count, epochs=300, report_delay=1)
# w2v_model.init_sims(replace=True)

# vector = w2v_model.wv['работа']
# print(vector)
!mkdir self-trained_word2vec
w2v_model.save('self-trained_word2vec/word2vec.model')

word_vectors = w2v_model.wv
X_train_w2v_self_trained = word_averaging_list(w2v_model, X_train_token)
X_test_w2v_self_trained = word_averaging_list(w2v_model, X_test_token)

In [4]:
print(datetime.datetime.now().strftime('%d-%B-%Y %H:%M:%S'))

26-May-2023 18:36:48


# FastText-признаки, обученные самостоятельно

## Обучение модели FastText на полученном наборе данных

In [15]:
ft_model = FastText(sentences=X_token, vector_size=300, window=5, min_count=1, workers=cpu_count, seed=random_state)
ft_model.build_vocab(X_token)
ft_model.train(X_token, total_examples=ft_model.corpus_count, epochs=300, report_delay=1)
# ft_model.init_sims(replace=True

# vector = ft_model.wv['работа']
# print(vector)
!mkdir self-trained_fasttext
ft_model.save('self-trained_fasttext/fasttext.model')

word_vectors = ft_model.wv
X_train_ft_self_trained = word_averaging_list(ft_model, X_train_token)
X_test_ft_self_trained = word_averaging_list(ft_model, X_test_token)

In [5]:
print(datetime.datetime.now().strftime('%d-%B-%Y %H:%M:%S'))

26-May-2023 18:36:52


# Word2Vec-признаки предобученные

## Предобработка текста

In [17]:
train = pd.read_csv(PATH + 'X_y_train.csv', sep=';')
test = pd.read_csv(PATH + 'X_y_test.csv', sep=';')

print("Processing input...", file=sys.stderr)
X_train_token_pre_trained = []
for input_line in train['Text']:
    res = unify_sym(input_line.strip())
    output = process(process_pipeline, text=res)
    X_train_token_pre_trained.append(output)


X_test_token_pre_trained = []
for input_line in test['Text']:
    res = unify_sym(input_line.strip())
    output = process(process_pipeline, text=res)
    X_test_token_pre_trained.append(output)

Processing input...


In [7]:
print(datetime.datetime.now().strftime('%d-%B-%Y %H:%M:%S'))

26-May-2023 18:37:00


## Загрузка предобученной модели Word2Vec

In [19]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format(PATH + 'word2vec/model.bin', encoding='utf-8', unicode_errors='ignore', binary=True)
w2v_model.fill_norms(force=True)

X_train_w2v_pre_trained = word_averaging_list_pre_trained(w2v_model, X_train_token_pre_trained)
X_test_w2v_pre_trained = word_averaging_list_pre_trained(w2v_model, X_test_token_pre_trained)

In [9]:
print(datetime.datetime.now().strftime('%d-%B-%Y %H:%M:%S'))

26-May-2023 18:37:10


# FastText-признаки предобученные

## Предобработка текста

In [21]:
train = pd.read_csv(PATH + 'X_y_train.csv', sep=';')
test = pd.read_csv(PATH + 'X_y_test.csv', sep=';')

print("Processing input...", file=sys.stderr)
X_train_token_pre_trained = []
for input_line in train['Text']:
    res = unify_sym(input_line.strip())
    output = process(process_pipeline, text=res, keep_pos=False)
    X_train_token_pre_trained.append(output)


X_test_token_pre_trained = []
for input_line in test['Text']:
    res = unify_sym(input_line.strip())
    output = process(process_pipeline, text=res, keep_pos=False)
    X_test_token_pre_trained.append(output)

Processing input...


In [8]:
print(datetime.datetime.now().strftime('%d-%B-%Y %H:%M:%S'))

26-May-2023 18:37:05


## Обучение модели FastText на полученном наборе данных

In [23]:
ft_model = gensim.models.KeyedVectors.load(PATH + 'fasttext/model.model')
ft_model.fill_norms(force=True)


# vector = ft_model[ft_model.key_to_index['работа']]
# print(vector)
# w2v_model.save('fasttext.model')


# word_vectors = w2v_model[:]
# X = word_averaging_list(ft_model, X_token)

X_train_ft_pre_trained = word_averaging_list_pre_trained(ft_model, X_train_token_pre_trained)
X_test_ft_pre_trained = word_averaging_list_pre_trained(ft_model, X_test_token_pre_trained)

## Обучение нейронных сетей

In [24]:

param_grid = {
#     "layers": [
#         [32, 16, 8, 4]
#     ],
    "embeddings": ['w2v', 'ft'],
    "pretrained": [False, True],
    "activation_fn": [F.relu, F.leaky_relu],
    "learning_rate": [0.001, 0.01],
    "epochs": [50, 10, 5],
    "optimizer": [optim.SGD, optim.AdamW],
    "batch_size": [2048],
    "layers_count": [3, 2, 1],
    "kernel_size": [3, 4, 5],
    "stride": [1, 2]
}

params_list = ParameterGrid(param_grid)
len(params_list)

1728

In [25]:
class Net(nn.Module):
    def __init__(self, device=None, layers_count=None, activation_function=None, kernel=None, stride=None):
        super().__init__()
        
        self.embedding_size = 300
        
        self.fa = activation_function
        
        self.kernel = kernel
        
        self.stride = stride
        
        self.layers = nn.ModuleList([])
        
        output_shape = 1
        
        for i in range(layers_count):
            self.layers.append(nn.Conv1d(in_channels = self.embedding_size, out_channels = self.embedding_size, kernel_size = self.kernel, stride = self.stride, padding = 5, device=device))
            output_shape = 1 + (output_shape + 2*5 - self.kernel) // self.stride
        
        
        self.fc = nn.Linear(output_shape * self.embedding_size, 2, device=device)
        
    def forward(self, x):

        x = x[:, :, None]
        
        for conv_layer in self.layers:
            x = conv_layer(x)
            x = self.fa(x)
        
        x = self.fc(x.reshape(x.shape[0], -1))
        
        return F.log_softmax(x, dim=-1)

In [26]:
from torchmetrics.classification import Accuracy, F1Score 
from tqdm import tqdm
from sklearn.model_selection import train_test_split

total = []

# accuracy_criterion = Accuracy(task='multiclass', num_classes=2, multidim_average='global', average='weighted').to(device)
f1_criterion = F1Score(task='multiclass', num_classes=2, multidim_average='global', average='weighted').to(device)

file = open('logs.txt', mode='w+')

for params in tqdm(params_list):
    
    learningRate = params['learning_rate']
    optimizer_type = params['optimizer']
    layers_count = params['layers_count']
    batch_size = params['batch_size']
    epochs = params['epochs']
    fa = params['activation_fn']
    kernel_size = params['kernel_size']
    stride = params['stride']
    embeddings = params['embeddings']
    pretrained = params['pretrained']
    

    net = Net(device=device, layers_count=layers_count, activation_function=fa, kernel=kernel_size, stride=stride).to(device)
    optimizer = optimizer_type(net.parameters(), lr=learningRate)
    
    XX_train, yy_train, XX_val, yy_val, XX_test, yy_test = [], [], [], [], [], []
    
    if embeddings == 'w2v':
        if pretrained:
            XX_train, XX_val, yy_train, yy_val = train_test_split(X_train_w2v_pre_trained, y_train, test_size=0.2, random_state=random_state)
            XX_test, yy_test = X_test_w2v_pre_trained, y_test
        else:
            XX_train, XX_val, yy_train, yy_val = train_test_split(X_train_w2v_self_trained, y_train, test_size=0.2, random_state=random_state)
            XX_test, yy_test = X_test_w2v_self_trained, y_test
    elif embeddings == 'ft':
        if pretrained:
            XX_train, XX_val, yy_train, yy_val = train_test_split(X_train_ft_pre_trained, y_train, test_size=0.2, random_state=random_state)
            XX_test, yy_test = X_test_ft_pre_trained, y_test
        else:
            XX_train, XX_val, yy_train, yy_val = train_test_split(X_train_ft_self_trained, y_train, test_size=0.2, random_state=random_state)
            XX_test, yy_test = X_test_ft_self_trained, y_test
    
    features_train = torch.tensor(XX_train, device=device, dtype=torch.float)
    targets_train = torch.tensor(yy_train, device=device, dtype=torch.int32)
    
    features_val = torch.tensor(XX_val, device=device, dtype=torch.float)
    targets_val = torch.tensor(yy_val, device=device, dtype=torch.int32)
    
    features_test = torch.tensor(XX_test, device=device, dtype=torch.float)
    targets_test = torch.tensor(yy_test, device=device, dtype=torch.int32)
    
    trainset = data_utils.TensorDataset(features_train, targets_train)
    valset = data_utils.TensorDataset(features_val, targets_val)
    testset = data_utils.TensorDataset(features_test, targets_test)
    
    train_loader = data_utils.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = data_utils.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_loader = data_utils.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    for epoch in range(epochs):
        net.train()
        
        for X, Y in train_loader:
            output = net(X)
            loss = F.cross_entropy(output, Y.long())
            
            net.zero_grad()
            
            loss.backward()
            optimizer.step()
            
    
    train_f1 = 0
    val_f1 = 0
    test_f1 = 0
    
    with torch.no_grad():
        net.eval()
        
        # evaluate the model on train
        targets = torch.tensor([], device=device)
        predictions = torch.tensor([], device=device)

        for X, Y in train_loader:
            output = net(X)
            
            targets = torch.cat((targets, Y))
            predictions = torch.cat((predictions, output.argmax(dim=-1)))
        
        train_f1 = f1_criterion(predictions, targets).item()

        # evaluate the model on validation
        targets = torch.tensor([], device=device)
        predictions = torch.tensor([], device=device)

        for X, Y in val_loader:
            output = net(X)
            
            targets = torch.cat((targets, Y))
            predictions = torch.cat((predictions, output.argmax(dim=-1)))
        
        val_f1 = f1_criterion(predictions, targets).item()

        # evaluate the model on test
        targets = torch.tensor([], device=device) 
        predictions = torch.tensor([], device=device)

        for X, Y in test_loader:
            output = net(X)
            
            targets = torch.cat((targets, Y))
            predictions = torch.cat((predictions, output.argmax(dim=-1)))
        
        test_f1 = f1_criterion(predictions, targets).item()
            
        
    
    total.append({
        'Модель': embeddings,
        'Предобученная': pretrained,
        'Функция активации': fa.__name__,
        'Размер ядра': kernel_size,
        "Отступ": stride,
        "Скорость обучения": learningRate,
        'Количество слоев': layers_count,
        'Batch_size': batch_size,
        'Оптимизатор': type(optimizer).__name__,
        'Количество эпох': epochs,
        'F1-train': round(train_f1, 2),
        'F1-val': round(val_f1, 2),
        'F1-test': round(test_f1, 2)
    })


100%|██████████| 1728/1728 [6:38:15<00:00, 13.83s/it]


In [10]:
print(datetime.datetime.now().strftime('%d-%B-%Y %H:%M:%S'))

26-May-2023 18:37:17


# Сводная таблица

In [28]:
pd.set_option('display.max_rows', None)
summary = pd.DataFrame.from_dict(total)
summary_sort = summary.sort_values(by='F1-val', ascending=False)
summary_sort

,Модель,Предобученная,Функция активации,Размер ядра,Отступ,Скорость обучения,Количество слоев,Batch_size,Оптимизатор,Количество эпох,F1-train,F1-val,F1-test
564,ft,False,relu,5,1,0.001,1,2048,AdamW,50,0.82,0.74,0.70
516,ft,False,relu,4,1,0.001,1,2048,AdamW,50,0.80,0.73,0.74
1429,ft,False,leaky_relu,5,2,0.001,1,2048,AdamW,50,0.76,0.73,0.74
1428,ft,False,leaky_relu,5,1,0.001,1,2048,AdamW,50,0.80,0.73,0.64
565,ft,False,relu,5,2,0.001,1,2048,AdamW,50,0.77,0.73,0.74
517,ft,False,relu,4,2,0.001,1,2048,AdamW,50,0.78,0.73,0.72
1380,ft,False,leaky_relu,4,1,0.001,1,2048,AdamW,50,0.79,0.73,0.68
476,ft,False,relu,3,1,0.010,1,2048,AdamW,50,0.84,0.73,0.64
468,ft,False,relu,3,1,0.001,1,2048,AdamW,50,0.79,0.73,0.68
1332,ft,False,leaky_relu,3,1,0.001,1,2048,AdamW,50,0.78,0.73,0.68
